## Importamos las librerías necesarias para nuestro tratado de datos

In [1]:
import pandas as pd
import numpy as np
import sidetable as stb
import re

# Eliminamos la limitación por defecto para las columnas y las filas
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Cargamos nuestro dataset

In [2]:
movies = pd.read_csv("data/movies.csv", index_col=0).reset_index()

In [3]:
movies.sample(5)

,Title,Rating,Year,Month,Certificate,Runtime,Directors,Stars,Genre,Filming_location,Budget,Income,Country_of_origin
713,The Revenant,8.0,2015,January,R,156,Alejandro G I rritu,"Leonardo DiCaprio, Tom Hardy, Will Poulter, Do...","Action, Adventure, Drama",Argentina,"$135,000,000","$532,950,503","Hong Kong, Taiwan, United States"
1332,District 9,7.9,2009,August,R,112,Neill Blomkamp,"Sharlto Copley, David James, Jason Cope, Natha...","Action, Sci-Fi, Thriller",South Africa,"$30,000,000","$210,888,950","South Africa, United States, New Zealand, Canada"
920,Oldboy,5.8,2013,November,R,104,Spike Lee,"Josh Brolin, Elizabeth Olsen, Samuel L Jackson...","Action, Drama, Mystery",USA,"$30,000,000","$5,186,767",United States
686,The Autopsy of Jane Doe,6.8,2016,December,R,86,Andr vredal,"Brian Cox, Emile Hirsch, Ophelia Lovibond, Mic...","Horror, Mystery, Thriller",UK,Unknown,"$6,170,043",United Kingdom
1758,Into the Blue,5.9,2005,September,PG-13,110,John Stockwell,"Paul Walker, Jessica Alba, Scott Caan, Ashley ...","Action, Adventure, Crime",Bahamas,"$50,000,000","$46,117,696",United States


A primera vista parece que el dataset está bastante limpio.

Antes de continuar vamos a prodecer a pasar los nombres de las columnas a minúsculas en vez de estar en Capitalize.

In [4]:
cols = [name.lower() for name in movies.columns ]

In [5]:
movies.columns = cols

### Leyenda de las columnas


|**COLUMNA**| **CONTENIDO**|
|-----------|---------------|
| title| Título de la película|
| rating| La puntuación que tiene en IMDB|
| year| Año en el que se estrenó la película|
| month| Mes de estreno|
| certificate| clasificación del publico al que está dirigida la película|
| runtime| Duración en minutos de la película|
| directors| Director de la película|
| stars| Actores principales de la película|
| genre| Género al que pertece la película|
| filming_location| Lugar donde se rodó la película|
| budget| Presupuesto para rodar la película|
| income| Ingresos obtenidos de la película|
| country_of_origin| País de origen de la película|

## Empezamos con un EDA básico

### Primero vemos si tenemos duplicados

In [6]:
movies.duplicated().sum()

0

### A continuación comprobamos el tipo de dato, tamaño del df y si tenemos duplicados

In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              2000 non-null   object 
 1   rating             1999 non-null   float64
 2   year               2000 non-null   int64  
 3   month              2000 non-null   object 
 4   certificate        1966 non-null   object 
 5   runtime            2000 non-null   object 
 6   directors          2000 non-null   object 
 7   stars              2000 non-null   object 
 8   genre              2000 non-null   object 
 9   filming_location   2000 non-null   object 
 10  budget             2000 non-null   object 
 11  income             2000 non-null   object 
 12  country_of_origin  2000 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 203.2+ KB


Tras una pequeña observación vemos que tenemos un total de 13 columnas por 2000 entradas y apenas tenemos duplicados

| COLUMNA| TIPO DE DATO | PRESENCIA DE NULOS |
|--------|--------------|--------------------|
| title| object - CORRECTO| NO| 
| rating| float - CORRECTO| SI| 
| year| int - CORRECTO| NO|
| month|object - CORRECTO| NO|
| certificate| object - CORRECTO| SI|
| runtime| object - INCORRECTO, debería ser INT| NO|
| directors| object - CORRECTO| NO|
| stars| object - CORRECTO| NO|
| genre| object - CORRECTO| NO|
|filming_location| object - CORRECTO| NO|
| budget| object - INCORRECTO, debería ser FLOAT| NO|
| income| object - INCORRECTO, debería ser FLOAT| NO|
| country_of_origin| object - CORRECTO| NO|

### Chequeamos en profundidad la cantidad exacta de nulos

In [8]:
movies.stb.missing()

,missing,total,percent
certificate,34,2000,1.70
rating,1,2000,0.05
title,0,2000,0.00
year,0,2000,0.00
month,0,2000,0.00
runtime,0,2000,0.00
directors,0,2000,0.00
stars,0,2000,0.00
genre,0,2000,0.00
filming_location,0,2000,0.00


Como ya habíamos visto tras el paso anterior solo tenemos nulos en dos columnas y sus números totales son muy pequeños. Más abajo estudiaremos si debemos hacer algún tratado especial.

## Antes de continuar procedemos a "limpiar" y cambiar el tipo de dato de las columnas:
 - runtime
 - budget
 - income


### Procedemos con a ver los valores únicos de la columna "runtime" para ver si nos puede dar una pista de porque no está en formato INT

In [9]:
movies["runtime"].value_counts()

104        50
100        49
105        48
97         48
109        47
102        47
107        45
117        45
98         44
101        42
115        42
103        42
99         42
106        41
118        41
113        40
92         39
116        39
94         38
90         38
119        38
112        37
110        36
95         36
108        36
93         36
111        35
124        34
123        34
96         34
91         31
120        28
114        28
130        28
121        28
129        27
122        26
132        25
127        24
88         22
134        22
126        21
86         21
125        21
128        20
131        19
85         19
89         19
135        19
138        18
87         16
137        16
140        16
139        14
133        14
136        13
143        12
141        12
148        12
144        11
84         10
146         9
152         9
147         8
83          7
151         6
154         6
158         6
149         5
142         5
165         5
82    

Vemos que tenemos dos entradas en nuestro dataset con el campo rellenado como "Unknown" . Así que al ser un número asequible de busquedas procedemos a filtrar el dataset para rellenarlos de manera manual.

In [10]:
movies[movies["runtime"]== "Unknown"]

,title,rating,year,month,certificate,runtime,directors,stars,genre,filming_location,budget,income,country_of_origin
848,Mantus,2.1,2014,2014,NaN,Unknown,Enzo Zelocchi,"Enzo Zelocchi, Miryam Negrin, Paul Gregory, Ma...","Comedy, Drama, Thriller",USA,"$250,000",Unknown,United States
1249,My Little Princess,9.6,2010,September,NaN,Unknown,Enzo Zelocchi,"Enzo Zelocchi, Charlotte Labadie, David M Edel...","Romance, Drama, Family",Unknown,$10,Unknown,United States


Tras una busqueda por internet descubrimos que la duraciones que faltan son de 100 y 21 minutos respectivamente.

In [11]:
movies.iloc[848,5]= 100
movies.iloc[1249,5]= 21

Comprobamos que las entradas tienen los nuevos regitros

In [12]:
movies.iloc[848]

title                                                           Mantus
rating                                                             2.1
year                                                              2014
month                                                             2014
certificate                                                        NaN
runtime                                                            100
directors                                                Enzo Zelocchi
stars                Enzo Zelocchi, Miryam Negrin, Paul Gregory, Ma...
genre                                          Comedy, Drama, Thriller
filming_location                                                   USA
budget                                                       $250,000 
income                                                         Unknown
country_of_origin                                        United States
Name: 848, dtype: object

In [13]:
movies.iloc[1249]

title                                               My Little Princess
rating                                                             9.6
year                                                              2010
month                                                        September
certificate                                                        NaN
runtime                                                             21
directors                                                Enzo Zelocchi
stars                Enzo Zelocchi, Charlotte Labadie, David M Edel...
genre                                           Romance, Drama, Family
filming_location                                               Unknown
budget                                                            $10 
income                                                         Unknown
country_of_origin                                        United States
Name: 1249, dtype: object

In [14]:
movies["runtime"] = movies["runtime"].astype(int)

### Seguimos con la columna "budget". Primero vemos sus valores únicos para valorar la limpieza.

In [15]:
movies["budget"].value_counts()

Unknown            304
$30,000,000         73
$40,000,000         65
$20,000,000         58
$25,000,000         54
$50,000,000         53
$150,000,000        50
$60,000,000         48
$15,000,000         41
$100,000,000        41
$200,000,000        39
$35,000,000         32
$75,000,000         31
$80,000,000         29
$5,000,000          28
$45,000,000         27
$55,000,000         24
$110,000,000        23
$70,000,000         22
$10,000,000         21
$90,000,000         21
$65,000,000         20
$12,000,000         20
$120,000,000        20
$85,000,000         19
$6,000,000          18
$130,000,000        18
$26,000,000         17
$18,000,000         17
$19,000,000         15
$125,000,000        15
$7,000,000          15
$38,000,000         15
$17,000,000         14
$175,000,000        14
$160,000,000        14
$2,000,000          14
$3,000,000          13
$32,000,000         12
$9,000,000          12
$250,000,000        12
$24,000,000         12
$28,000,000         12
$8,000,000 

Tras la observación de los valores únicos vemos que las entradas contienen "," y que todas están acompañadas del símbolo de la moneda que refleja la cantidad gastada en crear la película, esto "complicará la limpieza"  porque para poder contrastar bien los datos lo adecuado es que estén todos en la misma unidad numérica.

#### Lo primero que vamos hacer es eliminar las comas, ya que Python no interpreta con comas los números, y los espacios también.

In [16]:
movies["budget"] = movies["budget"].str.replace(",", "")
movies["budget"] = movies["budget"].str.replace(" ", "")

#### Lo siguiente que vamos hacer es identificar que tipo de símbolos tenemos como unidad de moneda

In [17]:
patron_caracteres = r"(\D+)" # Creamos un patron de regex para encontrar todos los símbolos que no sean numéricos
lista_vacia = [] # Y una lista vacía donde apendearemos los símbolos que encontremos

for valor in movies["budget"]:
    lista_vacia.append(re.findall(patron_caracteres, valor)[0])
        

In [18]:
caracteres_unicos = set(lista_vacia) # Convertimos a set para eliminar duplicados 
caracteres_unicos

{'$',
 'A$',
 'CA$',
 'CN¥',
 'DKK\xa0',
 'NOK\xa0',
 'SEK\xa0',
 'Unknown',
 '£',
 '¥',
 '₩',
 '€',
 '₹'}

De esta forma ya sabemos que monedas tenemos en la columna de "budget". Vamos a prodeceder a crear una función para automatizar el proceso de cambio de divisa.

Uno de los símbolos que identificamos es el won. Corea del Norte y Corea del Sur comparten moneda aunque no cambio de divisa, por lo que para asegurarnos, auqnue lo más probable es que estemos hablando de películas de corea del sur, vamos hacer una comprobación.



In [19]:
lista_won = ['₩10000000000', '₩12215500000', '₩4200000000' ]
movies[movies["budget"].isin(lista_won)]

,title,rating,year,month,certificate,runtime,directors,stars,genre,filming_location,budget,income,country_of_origin
604,The Handmaiden,8.1,2016,June,Not Rated,145,Park Chan wook,"Kim Min hee, Ha Jung woo, Cho Jin woong, Moon ...","Drama, Romance, Thriller",Japan,₩10000000000,"$37,854,655",South Korea
624,Train to Busan,7.6,2016,July,Not Rated,118,Sang ho Yeon,"Gong Yoo, Jung Yu mi, Ma Dong seok, Su an Kim","Action, Horror, Thriller",South Korea,₩10000000000,"$92,764,199",South Korea
1673,The Host,7.1,2006,March,R,120,Bong Joon Ho,"Song Kang ho, Byun Hee Bong, Park Hae il, Bae ...","Drama, Horror, Sci-Fi",South Korea,₩12215500000,"$89,433,436",South Korea
1782,Lady Vengeance,7.5,2005,July,R,115,Park Chan wook,"Nam mi Kang, Jeong nam Choi, Hye Sook Go, Bok ...","Crime, Drama, Thriller",South Korea,₩4200000000,"$23,834,149",South Korea


Efectivamente son películas que su país de origen pertenece a Corea del Sur.

Lo mismo ocurre con las tres naciones Escandinavas (Noruega,Dinamarca y Suecia), que comparten moneda, la corona, pero no cambio de divisa. No obstante estás están mejor identificadas y no hace falta ahcer comprobaciones.

Hay un símbolo que no identificamos, "A$", así que vamos a proceder a su búsqueda para ver si sus entradas nos dan alguna pista de que tipo de moneda es o si es solo un error.

In [20]:
lista_caracter_desconocido = ["A$3000000", "A$8240000"]
movies[movies["budget"].isin(lista_caracter_desconocido)]

,title,rating,year,month,certificate,runtime,directors,stars,genre,filming_location,budget,income,country_of_origin
1105,Sleeping Beauty,5.3,2011,December,TV-14,101,Julia Leigh,"Emily Browning, Rachael Blake, Ewen Leslie, Br...","Drama, Mystery, Romance",Australia,A$3000000,"$408,680",Australia
1390,Mary and Max,8.1,2009,March,Not Rated,92,Adam Elliot,"Toni Collette, Philip Seymour Hoffman, Eric Ba...","Animation, Comedy, Drama",Australia,A$8240000,"$1,740,429",Australia


Son dolares australianos.

Hacemos una búsqueda para ver cuantos países tienen como "Unknown" la entrada de la columna "budget"

In [21]:
len(movies[movies["budget"]== "Unknown"])

304

Como son más de 300 títulos con la entrada en la columna de "budget" como "Unknown" vamos a cambiar este valor por un Nan, que nos permitirá tener la columna en tipo de dato numérico y si el día de mañana tenemos oportunidad de rellenar esta información con un tratamiento para nulos puede ser relativamente "sencillo"

#### Como último paso para esta columna será hacer el cálculo de todos los presupuestos que no sean en dolares a esta moneda, ya que es la más internacional.


Los cambios de divisas se han realizado el 9 de marzo. En esa fecha los cambios de moneda están a:

|**MONEDA ORIGINAL**|**DOLARES**|
|-------------------|-----------|
| 1 libra | 1,19 $|
| 1 Yuan | 0.14 $|
| 1 euro | 1,06 $|
| 1 won(surcoreano) | 0.00076 $|
| 1 rupia | 0.012 $|
| 1 SEK(Suecia) | 0.093 $|
| 1 NOK(Noruega) | 0.094 $|
| 1 DKK(Dinamarca) | 0.014 $|
| 1 CA$(Canadienses) | 0.72 $|
| 1 A$(Australianos) | 0.66 $|




In [22]:
def cambio_divisa(col):
    dicc_divisas_valor = { '$': 1, 'A$': 0.66, 'CA$': 0.72 ,'CN¥': 0.14, 'DKK\xa0': 0.014, 'NOK\xa0': 0.094 , 'SEK\xa0': 0.093 , '£': 1.19, '¥': 0.14 ,'₩': 0.00076, '€': 1.06 , '₹': 0.012}
   

    patron_caracteres = r"(\D+)(\d+)" 

    if col == "Unknown":
        return np.nan
    else: 
        pass


 
    simbolo = re.findall(patron_caracteres, col)
    #print(simbolo)
    #print(simbolo[0][0])
    if simbolo[0][0] in dicc_divisas_valor.keys():

        num = int(simbolo[0][1])
        #print(num)
        cambio = num * dicc_divisas_valor[simbolo[0][0]]
        #print(cambio)
        return cambio
    
    


In [23]:
movies["dollar_budget"] = movies["budget"].apply(cambio_divisa)

In [24]:
movies.sample(2)

,title,rating,year,month,certificate,runtime,directors,stars,genre,filming_location,budget,income,country_of_origin,dollar_budget
764,Black Mass,6.9,2015,September,R,123,Scott Cooper,"Johnny Depp, Benedict Cumberbatch, Dakota John...","Biography, Crime, Drama",USA,$53000000,"$99,775,678","United States, United Kingdom",53000000.0
70,See How They Run,6.5,2022,September,PG-13,98,Tom George,"Kieran Hodgson, Pearl Chanda, Gregory Cox, Har...","Comedy, Mystery",UK,$40000000,"$21,966,634","United States, United Kingdom",40000000.0


Damos por finalizada la limpieza de la columna "budget" con la creación de la columna limpia "dollar_budget"

### Comenzamos la limpieza de la columna "income".

#### Chequeamos sus valores únicos para ver a que nos enfrentamos

In [25]:
movies["income"].value_counts()

Unknown            145
$2,267,946,983       1
$205,599,393         1
$71,585,235          1
$27,699,692          1
$443,140,005         1
$47,405,566          1
$28,190,603          1
$102,366,815         1
$61,808,775          1
$31,430,334          1
$54,004,950          1
$186,167,139         1
$122,444,772         1
$394,717             1
$219,103,655         1
$16,443,609          1
$55,720,772          1
$183,658,498         1
$6,577,779           1
$3,492               1
$91,509,154          1
$63,649,529          1
$2,425,535           1
$48,428,063          1
$13,657,649          1
$48,858,618          1
$68,777,554          1
$129,540,522         1
$62,357,900          1
$87,812,371          1
$36,348,784          1
$40,416,563          1
$108,902,486         1
$335,802,786         1
$1,149,138           1
$790,653,942         1
$163,728,902         1
$128,108,211         1
$226,837,760         1
$195,702,963         1
$585,796,247         1
$408,430,415         1
$39,628,044

Parece que esta columna será más sencilla ya que todas las entradas paraecen estar reflejadas en dolares, por lo que solo hay quemover el síbolo "$" y las comas.

En esta columna también tendremos valores "Unknown" que vamos a reemplazar por np.nan, para como en la columna anterior, permitir su conversión a tipo numérico y ademas de encontrar la información para rellenar será sencillo con algún método para tratar los nulos.

In [26]:
movies["income"] = movies["income"].str.replace("$", "")
movies["income"] = movies["income"].str.replace(",", "")

/tmp/ipykernel_249/875373714.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies["income"] = movies["income"].str.replace("$", "")


In [27]:
def unknown_nulos(col):
    if col == "Unknown":
        return np.nan
    else:
        return col

In [28]:
movies["income"] = movies["income"].apply(unknown_nulos)
movies["income"] = pd.to_numeric(movies["income"])

Comprobamos que las columnas tratadas han cambiado el tipo de dato correctamente

In [29]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              2000 non-null   object 
 1   rating             1999 non-null   float64
 2   year               2000 non-null   int64  
 3   month              2000 non-null   object 
 4   certificate        1966 non-null   object 
 5   runtime            2000 non-null   int64  
 6   directors          2000 non-null   object 
 7   stars              2000 non-null   object 
 8   genre              2000 non-null   object 
 9   filming_location   2000 non-null   object 
 10  budget             2000 non-null   object 
 11  income             1855 non-null   float64
 12  country_of_origin  2000 non-null   object 
 13  dollar_budget      1696 non-null   float64
dtypes: float64(3), int64(2), object(9)
memory usage: 218.9+ KB


In [30]:
movies.sample(5)

,title,rating,year,month,certificate,runtime,directors,stars,genre,filming_location,budget,income,country_of_origin,dollar_budget
916,We're the Millers,7.0,2013,August,R,110,Rawson Marshall Thurber,"Jason Sudeikis, Jennifer Aniston, Emma Roberts...","Comedy, Crime",USA,$37000000,269994119.0,"United States, United Kingdom",37000000.0
1482,Defiance,7.1,2008,January,R,137,Edward Zwick,"Daniel Craig, Liev Schreiber, Jamie Bell, Alex...","Action, Drama, History",Lithuania,$32000000,51262751.0,United States,32000000.0
715,Love the Coopers,5.8,2015,November,PG-13,107,Jessie Nelson,"Steve Martin, Diane Keaton, John Goodman, Ed H...","Comedy, Fantasy, Romance",USA,$17000000,42426912.0,United States,17000000.0
510,The Square,7.1,2017,August,R,151,Ruben stlund,"Claes Bang, Elisabeth Moss, Dominic West, Terr...","Comedy, Drama",Sweden,Unknown,9501021.0,"Sweden, Germany, France, Denmark",NaN
1931,The Last Samurai,7.8,2003,December,R,154,Edward Zwick,"Tom Cruise, Ken Watanabe, Billy Connolly, Will...","Action, Drama",Japan,$140000000,454627263.0,"United States, New Zealand, Japan",140000000.0


La columna original de "budget" la mantendremos, aunque la hayamos limpiado y sea la nueva columna de "dollar_budget" la que usemos para los siguientes analisis, porque para el "informe final" aun nos puede dar información util.

## Vemos las estadísticas descriptivas de las columnas

Chequeamos primero las estadísticas descriptivas de las columnas de tipo numérico.

In [31]:
movies.describe().T 

,count,mean,std,min,25%,50%,75%,max
rating,1999.0,6.666583e+00,9.117045e-01,1.9,6.15,6.7,7.300000e+00,9.600000e+00
year,2000.0,2.012500e+03,5.767723e+00,2003.0,2007.75,2012.5,2.017250e+03,2.022000e+03
runtime,2000.0,1.131650e+02,1.970820e+01,21.0,99.00,110.0,1.240000e+02,2.420000e+02
income,1855.0,1.887206e+08,2.774062e+08,305.0,24915810.00,90842646.0,2.204566e+08,2.922918e+09
dollar_budget,1696.0,6.118250e+07,6.073130e+07,10.0,17000000.00,40000000.0,8.500000e+07,3.560000e+08


Salvo por las columnas de "income" y "dollar_budget" en general hay muy poca diferencia entre mediana y media, por lo cual nos encontraremos pococ outliers en las demás columnas de tipo numérico. Además vemos que:

- En la columna "rating" la media de puntuación ronda los 6,6 puntos, vamos a tener muchas películas pasables pero no especialmente buenas según los usuarios.
- La distribución de películas por año debe ser bastante homogénea, ya que en la columna de "year" vemos que el registro más bajo es de 2003 y el más alto es de 2022, y la media está en 2012, prácticamente la mitad.
- En la columna "runtime" la distribución tb debe ser bastante homogénea ya que mediay mediana son muy similares.


Ahora chequeamos las estadísticas descriptivas de las columnas de tipo categórico.

In [32]:
movies.describe(include=object).T

,count,unique,top,freq
title,2000,1989,The Hunt,2
month,2000,14,November,260
certificate,1966,12,R,867
directors,2000,1082,Ridley Scott,13
stars,2000,1990,"Jennifer Lawrence, Josh Hutcherson, Liam Hemsw...",2
genre,2000,244,"Action, Adventure, Sci-Fi",92
filming_location,2000,97,USA,906
budget,2000,305,Unknown,304
country_of_origin,2000,406,United States,942


Tras este analisis vemos que :

- En la colummna "title" tenemos 2000 entradas pero de estas solo 1989 son valores únicos, así que habrá películas que compartan título. En top nos sale que el nombre que se repite más veces es "The Hunt", 2 veces, pero probablemente con el resto de nombres que se repitan lo harán el mismo número de veces.

- En la columna "month" vemos que tenemos 14 valores únicos, habrá que investigar eso por si hay errores tipográficos o algo. También vemos que el mes que más se repite es "November" con 260 entradas.

- En la columna "certificate" tenemos 12 categorías pero según el top de categorías que nos dice que la más repetida, con 867 entradas, es "R", que significa menores de 17 requiere de acompañante de padres o tutor. El sistema de categorías en el que se encuentra R consta de 5 categorías... habrá que revisar los valores únicos por si los certificados están mezclados o hay errores tipográficos.

- En la columna "directors" Ridley Scott se lleva el premio al director más repetido con 13 entradas en esta categoría.

- En la columna "stars" vemos que los actores que más se repiten son "Jennifer Lawrence, Josh Hutcherson, Liam Hemsw...	", el reparto de la película los juegos del hambre. Como las estrellas por película están todas juntas en la misma columna es dificil ver quien ha participado en más o menos películas. Habrá que tratar esta columna de alguna forma para poder aprovechar su información de mejor manera.

- En la columna "genre" vemos quela categoría más repetida es "Action, Adventure, Sci-Fi" con 92 entradas en esta categoría, pero pasa un poco lo mismo que con la columna anterior, al tener varias categorías en la misma columna es muy dificil que puedan coinicidir diferentes títulos por lo que habrá que tratarla para poder generalizar un poco más en las estadísiticas.

En el resto de columnas "filming_lacation", "budget" y "country_of_origin" no destacamos nada más que lo que se puede ver a primera vista. La mayoría de películas son y se han rodado en USA y la información "real" de budget la hemos visto en las estadísticas descriptivas anteriores.

## Decidimos separar los principales actores en columnas separadas

Lo primero que vamos hacer es comprobar que en la columna "stars" haya siempre el mismo número de nombres

In [33]:
lista_vacia = []

for x in movies["stars"]:
    lista_vacia.append(len(x.split(",")))

total_actores = set(lista_vacia)
total_actores

{3, 4}

Vemos que no, que tenemos entre 3 y 4 nombres en esa columna, asi que vamos a crear una función pra separar los nombres y ponerlos en columnas separadas. 

In [34]:
def sacar_actores(df,col):
    contador = 0
    
    for lista_actores in df[col]:
        lista_actores = lista_actores.split(",")
        if len(lista_actores) == 4:
            df.loc[contador,"star_1"] = lista_actores[0]
            df.loc[contador,"star_2"] = lista_actores[1].lstrip()
            df.loc[contador,"star_3"] = lista_actores[2].lstrip()
            df.loc[contador,"star_4"] = lista_actores[3].lstrip()
            contador += 1
        else :
            df.loc[contador,"star_1"] = lista_actores[0]
            df.loc[contador,"star_2"] = lista_actores[1].lstrip()
            df.loc[contador,"star_3"] = lista_actores[2].lstrip()
            contador += 1
        
        

        #Vamos a tener que poner un contador y hacer que ponga un iloc para ir apendeando

In [35]:
sacar_actores(movies, "stars")

_______________

Descubrimos en el proceso un juego divertido, ver los nombres de los actores e intentar adivinar la película.

In [36]:
#movies.iloc[90, 7]

In [37]:
#movies.iloc[90,0]

_____________

In [38]:
movies.sample(5)

,title,rating,year,month,certificate,runtime,directors,stars,genre,filming_location,budget,income,country_of_origin,dollar_budget,star_1,star_2,star_3,star_4
526,Thor: Ragnarok,7.9,2017,November,PG-13,130,Taika Waititi,"Chris Hemsworth, Tom Hiddleston, Cate Blanchet...","Action, Adventure, Comedy",Australia,$180000000,853983879.0,"United States, Australia",180000000.0,Chris Hemsworth,Tom Hiddleston,Cate Blanchett,Mark Ruffalo
626,Moana,7.6,2016,November,PG,107,"Ron Clements, John Musker, Don Hall, Chris Wil...","Auli i Cravalho, Dwayne Johnson, Rachel House,...","Animation, Adventure, Comedy",Polynesia,$150000000,665069139.0,United States,150000000.0,Auli i Cravalho,Dwayne Johnson,Rachel House,Temuera Morrison
129,Home Sweet Home Alone,3.6,2021,November,PG,93,Dan Mazer,"Ellie Kemper, Rob Delaney, Archie Yates, Aisli...","Comedy, Crime, Family",Canada,$15000000,NaN,United States,15000000.0,Ellie Kemper,Rob Delaney,Archie Yates,Aisling Bea
1584,Surf's Up,6.7,2007,June,PG,85,"Ash Brannon, Chris Buck","Shia LaBeouf, Zooey Deschanel, Jon Heder, Jeff...","Animation, Adventure, Comedy",USA,$100000000,152005713.0,United States,100000000.0,Shia LaBeouf,Zooey Deschanel,Jon Heder,Jeff Bridges
219,The Croods: A New Age,6.9,2020,November,PG,95,Joel Crawford,"Nicolas Cage, Emma Stone, Ryan Reynolds, Cathe...","Animation, Adventure, Comedy",Unknown,$65000000,215905815.0,United States,65000000.0,Nicolas Cage,Emma Stone,Ryan Reynolds,Catherine Keener


## Separamos la columna "genre"

Lo primero que vamos hacer es contabilizar cuantos generos hay exactamente

In [39]:
lista_vacia = []

for x in movies["genre"]:
    x = x.split(",")
    
    for y in x:
       
        lista_vacia.append(y.lstrip())

total_generos = set(lista_vacia)
total_generos


{'Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Thriller',
 'War',
 'Western'}

Viendo los resultados sacamos que en la columna "genre" tenemos mezclados conceptos como género, temática y técnica.

Podríamos clasificar los resultados de la siguiente manera:

|CATEGORÍA  | GENERO |  TEMÁTICA  |  TÉCNICA |
|-----------|--------|------------|----------|
|'Action'   | *  |            |          |
| 'Adventure'| * |            |          | 
|'Animation' |       |            |  *   |
| 'Biography'|       | *      |          |
| 'Comedy'   | * |            |          | 
| 'Crime'   | *  |            |          |
| 'Drama'    | * |            |          | 
| 'Family'|          |  *     |          |
| 'Fantasy'|         |     *  |          | 
| 'History'|         |  *     |          |  
| 'Horror'|  *   |            |          |
| 'Music'|           | *      |          |
| 'Musical'|         | *      |          |
| 'Mystery'|  *  |            |          | 
| 'Romance'|  *  |            |          |
| 'Sci-Fi'|          | *      |          |
| 'Sport'|           |    *   |          |
| 'Thriller'|  * |            |          |
| 'War'|             |  *     |          |
| 'Western'|         |  *     |          |

Sacamos que: 

- En género tenemos 9 categorías que podrían ser 8 ya que podríamos englobar thriller y mystery como la misma categoría.
- En temática tenemos 10 que pueden ser 9 ya que music y musical también se pueden englobar en la misma categoría.
- Como técnica solo tenemos animación.

In [40]:
movies["genre"].value_counts()

Action, Adventure, Sci-Fi       92
Animation, Adventure, Comedy    77
Comedy, Drama, Romance          76
Drama                           65
Action, Adventure, Fantasy      59
Comedy                          57
Comedy, Romance                 56
Action, Adventure, Comedy       55
Drama, Romance                  53
Comedy, Drama                   50
Action, Crime, Drama            43
Crime, Drama, Thriller          42
Action, Crime, Thriller         41
Crime, Drama, Mystery           40
Horror, Mystery, Thriller       38
Action, Adventure, Drama        37
Action, Comedy, Crime           35
Action, Thriller                34
Biography, Drama, History       27
Horror, Thriller                25
Drama, Horror, Mystery          24
Adventure, Comedy, Family       23
Drama, Mystery, Thriller        23
Horror                          22
Animation, Action, Adventure    21
Drama, Thriller                 21
Biography, Crime, Drama         21
Adventure, Family, Fantasy      20
Action, Adventure, T

vamos a chequear cuantas categorías tenemos por fila

In [41]:
lista_vacia = []

for x in movies["genre"]:
    lista_vacia.append(len(x.split(",")))

total_num_generos = set(lista_vacia)
total_num_generos

{1, 2, 3}

Vamos a tratar la técnica, género y temática cada una por separado.

Lo primero que haremos ya que será lo más sencillo es crear una columna nueva para identificar las películas que sean de animación y las que no.

In [42]:
def animacion(col):
    col = col.split(",")
    for palabra in col:
        palabra = palabra.lstrip()
        if palabra == "Animation":
            return "Y"
        else:
            return "N"

In [43]:
movies["animation"] = movies["genre"].apply(animacion)

A continuación vamos a clasificar el género creando una columna por cada entrada de esta categoría.

In [44]:
def clasificar_generos(df,col):    

    
    lista_genero = ['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Horror','Romance', 'Thriller', 'Mystery' ]  
                   
                      
    contador_fila = 0

    for frase in df[col]:
        
        frase = frase.split(", ")
        #print(frase)
        for palabra in frase:
                #palabra = palabra #.lstrip()
                if palabra in lista_genero:
                        
                    df.loc[contador_fila,"genre_" + str(frase.index(palabra))] = palabra

                else:
                     pass

        contador_fila += 1



In [45]:
clasificar_generos(movies,"genre")

Hacemos lo mismo para las temáticas

In [46]:
def clasificar_tematicas(df,col):    

    
    lista_tematica = ['Biography','Family','Fantasy','History', 'Musical', 'Sci-Fi', 'Sport', 'War', 'Western', 'Music'] 
                   
                      
    contador_fila = 0

    for frase in df[col]:
        
        frase = frase.split(", ")
        #print(frase)
        for palabra in frase:
                #palabra = palabra #.lstrip()
                if palabra in lista_tematica:
                        
                    df.loc[contador_fila,"theme_" + str(frase.index(palabra))] = palabra

                else:
                     pass

        contador_fila += 1


In [47]:
clasificar_tematicas(movies,"genre")

In [50]:
movies.head(20)

,title,rating,year,month,certificate,runtime,directors,stars,genre,filming_location,budget,income,country_of_origin,dollar_budget,star_1,star_2,star_3,star_4,animation,genre_0,genre_1,genre_2,theme_2,theme_1,theme_0
0,Avatar: The Way of Water,7.8,2022,December,PG-13,192,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...","Action, Adventure, Fantasy",New Zealand,$350000000,2.267947e+09,United States,350000000.0,Sam Worthington,Zoe Saldana,Sigourney Weaver,Stephen Lang,N,Action,Adventure,NaN,Fantasy,NaN,NaN
1,Guillermo del Toro's Pinocchio,7.6,2022,December,PG,117,"Guillermo del Toro, Mark Gustafson","Ewan McGregor, David Bradley, Gregory Mann, Bu...","Animation, Drama, Family",USA,$35000000,1.089670e+05,"United States, Mexico, France",35000000.0,Ewan McGregor,David Bradley,Gregory Mann,Burn Gorman,Y,NaN,Drama,NaN,Family,NaN,NaN
2,Bullet Train,7.3,2022,August,R,127,David Leitch,"Brad Pitt, Joey King, Aaron Taylor Johnson, Br...","Action, Comedy, Thriller",Japan,$85900000,2.392686e+08,"Japan, United States",85900000.0,Brad Pitt,Joey King,Aaron Taylor Johnson,Brian Tyree Henry,N,Action,Comedy,Thriller,NaN,NaN,NaN
3,The Banshees of Inisherin,7.8,2022,November,R,114,Martin McDonagh,"Colin Farrell, Brendan Gleeson, Kerry Condon, ...","Comedy, Drama",Ireland,Unknown,1.972082e+07,"Ireland, United Kingdom, United States",NaN,Colin Farrell,Brendan Gleeson,Kerry Condon,Pat Shortt,N,Comedy,Drama,NaN,NaN,NaN,NaN
4,M3gan,6.4,2022,December,PG-13,102,Gerard Johnstone,"Jenna Davis, Amie Donald, Allison Williams, Vi...","Horror, Sci-Fi, Thriller",New Zealand,$12000000,1.712539e+08,United States,12000000.0,Jenna Davis,Amie Donald,Allison Williams,Violet McGraw,N,Horror,NaN,Thriller,NaN,Sci-Fi,NaN
5,Emancipation,6.1,2022,December,R,132,Antoine Fuqua,"Will Smith, Ben Foster, Charmaine Bingwa, Gilb...","Action, Thriller",Unknown,$120000000,NaN,United States,120000000.0,Will Smith,Ben Foster,Charmaine Bingwa,Gilbert Owuor,N,Action,Thriller,NaN,NaN,NaN,NaN
6,Amsterdam,6.1,2022,October,R,134,David O Russell,"Christian Bale, Margot Robbie, John David Wash...","Comedy, Drama, History",USA,$80000000,3.124581e+07,"United States, Japan",80000000.0,Christian Bale,Margot Robbie,John David Washington,Alessandro Nivola,N,Comedy,Drama,NaN,History,NaN,NaN
7,Violent Night,6.9,2022,December,R,112,Tommy Wirkola,"David Harbour, John Leguizamo, Beverly D Angel...","Action, Comedy, Crime",Canada,$20000000,5.959546e+07,"United States, Canada",20000000.0,David Harbour,John Leguizamo,Beverly D Angelo,Alex Hassell,N,Action,Comedy,Crime,NaN,NaN,NaN
8,The Whale,7.8,2022,December,R,117,Darren Aronofsky,"Brendan Fraser, Sadie Sink, Ty Simpkins, Hong ...",Drama,USA,$10000000,3.110431e+07,United States,10000000.0,Brendan Fraser,Sadie Sink,Ty Simpkins,Hong Chau,N,Drama,NaN,NaN,NaN,NaN,NaN
9,The Fabelmans,7.6,2022,November,PG-13,151,Steven Spielberg,"Michelle Williams, Gabriel LaBelle, Paul Dano,...",Drama,USA,$40000000,9.500361e+06,United States,40000000.0,Michelle Williams,Gabriel LaBelle,Paul Dano,Judd Hirsch,N,Drama,NaN,NaN,NaN,NaN,NaN


In [49]:
def clasificar_gerneros(col):

    dicc_clasificaiones = {"class_genre": ['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Horror','Romance', 'Thriller'], #'Mystery'
                           "theme": ['Biography','Family','Fantasy','History', 'Musical', 'Sci-Fi', 'Sport', 'War', 'Western'], #'Music'
                           "technique" : 'Animation'
                           }
    col = col.split(",").lstrip()
    if len(col) > 1:
        for palabra in col:
            if palabra == "Mystery":
                return "Thriller"
            elif palabra == 
    eli


SyntaxError: invalid syntax (542074707.py, line 12)

## Chequeamos los valores únicos